In [13]:
# Abrindo os arquivos e formatando 
from sklearn.model_selection import train_test_split
from tensorflow import keras
import pandas as pd
import tensorflow as tf
import numpy as np

data = pd.read_csv('data.csv', sep = ',')

# Separando atributos e resultados
x = data.drop(columns = ['letter'])
y = data['letter']

In [14]:
# Normalizando
from sklearn.preprocessing import MinMaxScaler
from pickle import dump

normalizer = MinMaxScaler()
normalizerModel = normalizer.fit(x)
dump(normalizerModel, open('normalizerModel.pkl', 'wb')) 

x_normalized = normalizer.fit_transform(x)
y = pd.DataFrame(y)

In [15]:
# Balanceando
from imblearn.over_sampling import SMOTE
resampler = SMOTE(k_neighbors=4)
x_balanced, y_balanced = resampler.fit_resample(x_normalized, y)

In [16]:
# Separando dados para atributos e treinamento
x_train, x_test, y_train, y_test = train_test_split(x_balanced, y_balanced, test_size=0.1)

In [17]:
# Verificando os dados
print("Quantidade de Dados de Treino:" ,len(x_train))
print("Quantidade de Dados de Teste:" ,len(x_test))

print("Quantidade de Dados de Treino e dos atributos:" ,x_train.shape)
print("Quantidade de Dados de Teste e dos atributos" ,x_test.shape)

print("Quantidade de Saidas de Treino e coluna:" ,y_train.shape)
print("Quantidade de Saidas de Teste e coluna:" ,y_test.shape)

print("min: ", y_train.min())
print("max: ", y_test.max())

Quantidade de Dados de Treino: 7228
Quantidade de Dados de Teste: 804
Quantidade de Dados de Treino e dos atributos: (7228, 42)
Quantidade de Dados de Teste e dos atributos (804, 42)
Quantidade de Saidas de Treino e coluna: (7228, 1)
Quantidade de Saidas de Teste e coluna: (804, 1)
min:  letter    0
dtype: int64
max:  letter    3
dtype: int64


In [18]:
# Criando o modelo da rede neural
modelo = keras.Sequential([
  keras.layers.Dense(250, activation=tf.nn.relu, input_shape =(42, )),
  keras.layers.Dense(150, activation=tf.nn.relu),
  keras.layers.Dense(110, activation=tf.nn.relu),
  keras.layers.Dense(50, activation=tf.nn.relu),
  keras.layers.Dense(80, activation=tf.nn.relu),
  keras.layers.Dense(4, activation=tf.nn.softmax)
])
modelo.compile(loss ='sparse_categorical_crossentropy', optimizer ='adam', metrics =['accuracy'])

In [19]:
# Treinando o modelo
x = modelo.fit(x_train, y_train, epochs = 150, batch_size = 64)
test_loss, test_accuracy = modelo.evaluate(x_test, y_test)

Epoch 1/150
113/113 [==============================] - 1s 2ms/step - loss: 0.4452 - accuracy: 0.8813
Epoch 2/150
113/113 [==============================] - 0s 2ms/step - loss: 0.0213 - accuracy: 0.9974
Epoch 3/150
113/113 [==============================] - 0s 2ms/step - loss: 0.0341 - accuracy: 0.9913
Epoch 4/150
113/113 [==============================] - 0s 2ms/step - loss: 0.0117 - accuracy: 0.9989
Epoch 5/150
113/113 [==============================] - 0s 2ms/step - loss: 0.0132 - accuracy: 0.9985
Epoch 6/150
113/113 [==============================] - 0s 2ms/step - loss: 0.0171 - accuracy: 0.9975
Epoch 7/150
113/113 [==============================] - 0s 2ms/step - loss: 0.0104 - accuracy: 0.9989
Epoch 8/150
113/113 [==============================] - 0s 2ms/step - loss: 0.0113 - accuracy: 0.9988
Epoch 9/150
113/113 [==============================] - 0s 2ms/step - loss: 0.0124 - accuracy: 0.9986
Epoch 10/150
113/113 [==============================] - 0s 2ms/step - loss: 0.0118 - accura

In [20]:
# Testando o modelo da rede neural com dados de validação
import HandTrackingModule as htm
from PIL import Image
import cv2
import numpy as np
formattedLm = []

img = Image.open('./ValidateImages/B1.png')
detector = htm.handDetector()

formattedImg = detector.formatImage(img)

img = detector.findHands(formattedImg)
lmList = detector.findPosition(img)

for lm in lmList:
  formattedLm.append(lm[1])
  formattedLm.append(lm[2])

x_normalized = pd.DataFrame(formattedLm).transpose()
x_normalized = normalizer.transform(x_normalized)
predict = modelo.predict(x_normalized)

print('Valor inferido: ', np.argmax(predict))

1/1 [==============================] - 0s 55ms/step
Valor inferido:  3


c:\Users\Administrador\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [21]:
alfabeto = ['A','B','C','D']
import cv2
import numpy as np
from pickle import load
import HandTrackingModule as htm
import pandas as pd
normalizer = load(open('normalizerModel.pkl', 'rb'))
detector = htm.handDetector()
cap = cv2.VideoCapture(0)
formattedLm = []
while True:
    success, img = cap.read()
    img = detector.findHands(img, draw=True)
    lmList = detector.findPositionWithoudId(img, draw=False)
    k = cv2.waitKey(1)
    if(len(lmList) != 0):
        x_normalized = pd.DataFrame(lmList).transpose()
        x_normalized = normalizer.transform(x_normalized)
        predict = modelo.predict(x_normalized, verbose =0)
        cv2.putText(img, alfabeto[int(np.argmax(predict))], (10, 70), cv2.FONT_HERSHEY_PLAIN, 3,(255, 0, 255), 3)
        cv2.putText(img, 'Acc: ' + str(predict[0][np.argmax(predict)]), (80, 70), cv2.FONT_HERSHEY_PLAIN, 3,(255, 0, 255), 3)

    if k%256 == 27:
        print("Escape hit, closing...")
        break
    cv2.imshow("Image", img)
cap.release()
cv2.destroyAllWindows()

c:\Users\Administrador\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\Administrador\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\Administrador\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\Administrador\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\Administrador\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\Administrador\anaconda3\lib\site-packages\sklearn\base.py:450

Escape hit, closing...
